In [11]:
import pandas as pd
import numpy as np
import requests

In [12]:
from IPython.display import display, Markdown, Latex
display(Markdown('*some markdown* $\phi$'))

*some markdown* $\phi$

In [13]:
##First 4 GW of 16-17
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw20.csv"]

##Next 6 GW of 16-17
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/gws/gw26.csv"]

In [14]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]


url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2016-17/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = 'utf-8', error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()

print(id_list[id_list["id"] == 482])

    first_name second_name   id
481  Abdoulaye    Doucouré  482


In [15]:
fix_diff = pd.read_excel('A:\Documents\EPL_Fixture_Difficulty.xlsx',sheet_name='16-17', index_col=0)
fix_diff = pd.DataFrame(fix_diff)
fix_diff.head

df_16_17 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_16_17 = df_16_17.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_16_17.dtypes
df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_16_17 = df_16_17.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_16_17_agg = df_16_17.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})
df_16_17_agg['round'] = max(df_16_17_agg['round'])
##Get Last Value for players in GW range
df_16_17_last = df_16_17.groupby('name',as_index=False).last()
df_16_17_last = df_16_17_last[['name','element','creativity','ict_index','influence','threat','value']]

df_16_17_first = df_16_17.groupby('name',as_index=False).first()
# df_16_17_first = df_16_17_first[['name','id']]

df_16_17 = df_16_17_agg.merge(df_16_17_last, left_on='name', right_on='name')
df_16_17 = df_16_17.merge(df_n6, left_on='name', right_on='name')
df_16_17['perc_min_played'] = df_16_17['minutes']/(df_16_17['round']*90)
df_16_17 = df_16_17.drop(columns=['minutes','round'], axis=0)

df_16_17.count()

name                      624
assists                   624
bonus                     624
bps                       624
clean_sheets              624
goals_conceded            624
goals_scored              624
own_goals                 624
penalties_missed          624
penalties_saved           624
red_cards                 624
saves                     624
selected                  624
total_points              624
transfers_balance         624
was_home                  624
yellow_cards              624
opponent_difficulty       624
element                   624
creativity                624
ict_index                 624
influence                 624
threat                    624
value                     624
n6_total_points           624
n6_opponent_difficulty    624
perc_min_played           624
dtype: int64

In [16]:
new = df_16_17["name"].str.split("_", expand = True) 

df_16_17["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_16_17["second_name"]= new[1] 

df_16_17.head()

df_16_17 = pd.merge(df_16_17, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_16_17 = df_16_17.drop(columns=['first_name','second_name'], axis=0)
df_16_17 = df_16_17.merge(players, left_on='id', right_on='id')

df_16_17.count()

name                      624
assists                   624
bonus                     624
bps                       624
clean_sheets              624
goals_conceded            624
goals_scored              624
own_goals                 624
penalties_missed          624
penalties_saved           624
red_cards                 624
saves                     624
selected                  624
total_points              624
transfers_balance         624
was_home                  624
yellow_cards              624
opponent_difficulty       624
element                   624
creativity                624
ict_index                 624
influence                 624
threat                    624
value                     624
n6_total_points           624
n6_opponent_difficulty    624
perc_min_played           624
id                        624
element_type              624
dtype: int64

In [17]:
##First 4 GW of 17-18
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw20.csv"]

##Next 6 GW of 17-18
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/gws/gw26.csv"]

In [18]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]

url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2017-18/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = 'utf-8', error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()

print(id_list[id_list["id"] == 482])

    first_name second_name   id
180      Jairo   Riedewald  482


In [19]:
fix_diff = pd.read_excel('A:\Documents\EPL_Fixture_Difficulty.xlsx',sheet_name='17-18', index_col=0)
fix_diff = pd.DataFrame(fix_diff)
fix_diff.head


df_17_18 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_17_18 = df_17_18.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_17_18.dtypes
df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_17_18 = df_17_18.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_17_18_agg = df_17_18.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})
df_17_18_agg['round'] = max(df_17_18_agg['round'])
##Get Last Value for players in GW range
df_17_18_last = df_17_18.groupby('name',as_index=False).last()
df_17_18_last = df_17_18_last[['name','element','creativity','ict_index','influence','threat','value']]

df_17_18_first = df_17_18.groupby('name',as_index=False).first()
# df_17_18_first = df_17_18_first[['name','id']]

df_17_18 = df_17_18_agg.merge(df_17_18_last, left_on='name', right_on='name')
df_17_18 = df_17_18.merge(df_n6, left_on='name', right_on='name')
df_17_18['perc_min_played'] = df_17_18['minutes']/(df_17_18['round']*90)
df_17_18 = df_17_18.drop(columns=['minutes','round'], axis=0)

df_17_18.count()

name                      586
assists                   586
bonus                     586
bps                       586
clean_sheets              586
goals_conceded            586
goals_scored              586
own_goals                 586
penalties_missed          586
penalties_saved           586
red_cards                 586
saves                     586
selected                  586
total_points              586
transfers_balance         586
was_home                  586
yellow_cards              586
opponent_difficulty       586
element                   586
creativity                586
ict_index                 586
influence                 586
threat                    586
value                     586
n6_total_points           586
n6_opponent_difficulty    586
perc_min_played           586
dtype: int64

In [20]:
new = df_17_18["name"].str.split("_", expand = True) 

df_17_18["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_17_18["second_name"]= new[1] 

df_17_18.head()

df_17_18 = pd.merge(df_17_18, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_17_18 = df_17_18.drop(columns=['first_name','second_name'], axis=0)
df_17_18 = df_17_18.merge(players, left_on='id', right_on='id')

df_17_18.count()



name                      585
assists                   585
bonus                     585
bps                       585
clean_sheets              585
goals_conceded            585
goals_scored              585
own_goals                 585
penalties_missed          585
penalties_saved           585
red_cards                 585
saves                     585
selected                  585
total_points              585
transfers_balance         585
was_home                  585
yellow_cards              585
opponent_difficulty       585
element                   585
creativity                585
ict_index                 585
influence                 585
threat                    585
value                     585
n6_total_points           585
n6_opponent_difficulty    585
perc_min_played           585
id                        585
element_type              585
dtype: int64

In [21]:
##First 4 GW of 2018-19
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw20.csv"]

##Next 6 GW of 18-19
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/gws/gw26.csv"]


In [22]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]

url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2018-19/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = 'utf-8', error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()

print(id_list[id_list["id"] == 482])

    first_name second_name   id
451     Fabian       Schär  482


In [23]:
fix_diff = pd.read_excel('A:\Documents\EPL_Fixture_Difficulty.xlsx',sheet_name='18-19', index_col=0)
fix_diff = pd.DataFrame(fix_diff)
fix_diff.head


df_18_19 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_18_19 = df_18_19.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_18_19.dtypes
df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_18_19 = df_18_19.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_18_19_agg = df_18_19.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})
df_18_19_agg['round'] = max(df_18_19_agg['round'])
##Get Last Value for players in GW range
df_18_19_last = df_18_19.groupby('name',as_index=False).last()
df_18_19_last = df_18_19_last[['name','element','creativity','ict_index','influence','threat','value']]
	
df_18_19_first = df_18_19.groupby('name',as_index=False).first()
# df_18_19_first = df_18_19_first[['name','id']]

df_18_19 = df_18_19_agg.merge(df_18_19_last, left_on='name', right_on='name')
df_18_19 = df_18_19.merge(df_n6, left_on='name', right_on='name')
df_18_19['perc_min_played'] = df_18_19['minutes']/(df_18_19['round']*90)
df_18_19 = df_18_19.drop(columns=['minutes','round'], axis=0)


df_18_19.count()


name                      568
assists                   568
bonus                     568
bps                       568
clean_sheets              568
goals_conceded            568
goals_scored              568
own_goals                 568
penalties_missed          568
penalties_saved           568
red_cards                 568
saves                     568
selected                  568
total_points              568
transfers_balance         568
was_home                  568
yellow_cards              568
opponent_difficulty       568
element                   568
creativity                568
ict_index                 568
influence                 568
threat                    568
value                     568
n6_total_points           568
n6_opponent_difficulty    568
perc_min_played           568
dtype: int64

In [24]:
new = df_18_19["name"].str.split("_", expand = True) 

df_18_19["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_18_19["second_name"]= new[1] 

df_18_19.head()

df_18_19 = pd.merge(df_18_19, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_18_19 = df_18_19.drop(columns=['first_name','second_name'], axis=0)
df_18_19 = df_18_19.merge(players, left_on='id', right_on='id')

df_18_19.count()


name                      568
assists                   568
bonus                     568
bps                       568
clean_sheets              568
goals_conceded            568
goals_scored              568
own_goals                 568
penalties_missed          568
penalties_saved           568
red_cards                 568
saves                     568
selected                  568
total_points              568
transfers_balance         568
was_home                  568
yellow_cards              568
opponent_difficulty       568
element                   568
creativity                568
ict_index                 568
influence                 568
threat                    568
value                     568
n6_total_points           568
n6_opponent_difficulty    568
perc_min_played           568
id                        568
element_type              568
dtype: int64

In [25]:
##First 4 GW of 2019-20
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw20.csv"]

##Next 6 GW of 19-20
url2 = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw21.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw22.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw23.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw24.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw25.csv",
       "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/gws/gw26.csv"]


In [26]:
url3 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/players_raw.csv"
players = pd.read_csv(url3, encoding = "ISO-8859-1", error_bad_lines=False)
players = pd.DataFrame(players)
players = players[['element_type','id']]


url4 = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2019-20/player_idlist.csv"
id_list = pd.read_csv(url4, encoding = "ISO-8859-1", error_bad_lines=False)
id_list = pd.DataFrame(id_list)
# id_list.head()
players.head()

,element_type,id
0,2,1
1,2,2
2,2,3
3,2,4
4,2,5


In [27]:
fix_diff = pd.read_excel('A:\Documents\EPL_Fixture_Difficulty.xlsx',sheet_name='19-20', index_col=0)
fix_diff = pd.DataFrame(fix_diff)
fix_diff.head

df_19_20 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_19_20 = df_19_20.append(df)
    
df_n6 = pd.DataFrame()
for data in url2:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_n6 = df_n6.append(df)

df_19_20.dtypes
df_n6 = df_n6.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_n6 = df_n6.groupby('name', as_index=False).agg({'total_points':'sum',
                                                  'opponent_difficulty':'mean'})
df_n6.rename(columns={'total_points':'n6_total_points','opponent_difficulty':'n6_opponent_difficulty'}, inplace=True)
df_19_20 = df_19_20.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')

df_19_20_agg = df_19_20.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})

df_19_20_agg['round'] = max(df_19_20_agg['round'])
##Get Last Value for players in GW range
df_19_20_last = df_19_20.groupby('name',as_index=False).last()
df_19_20_last = df_19_20_last[['name','element','creativity','ict_index','influence','threat','value']]
	
df_19_20_first = df_19_20.groupby('name',as_index=False).first()
# df_19_20_first = df_19_20_first[['name','id']]

df_19_20 = df_19_20_agg.merge(df_19_20_last, left_on='name', right_on='name')
df_19_20 = df_19_20.merge(df_n6, left_on='name', right_on='name')
df_19_20['perc_min_played'] = df_19_20['minutes']/(df_19_20['round']*90)
df_19_20 = df_19_20.drop(columns=['minutes','round'], axis=0)

df_19_20.count()

name                      584
assists                   584
bonus                     584
bps                       584
clean_sheets              584
goals_conceded            584
goals_scored              584
own_goals                 584
penalties_missed          584
penalties_saved           584
red_cards                 584
saves                     584
selected                  584
total_points              584
transfers_balance         584
was_home                  584
yellow_cards              584
opponent_difficulty       584
element                   584
creativity                584
ict_index                 584
influence                 584
threat                    584
value                     584
n6_total_points           584
n6_opponent_difficulty    584
perc_min_played           584
dtype: int64

In [28]:
new = df_19_20["name"].str.split("_", expand = True) 

df_19_20["first_name"]= new[0] 
  
# making separate last name column from new data frame 
df_19_20["second_name"]= new[1] 

df_19_20.head()

df_19_20 = pd.merge(df_19_20, id_list,  how='left', left_on=['first_name','second_name'], right_on=['first_name','second_name'])
df_19_20 = df_19_20.drop(columns=['first_name','second_name'], axis=0)
df_19_20 = df_19_20.merge(players, left_on='id', right_on='id')

df_19_20.count()

name                      584
assists                   584
bonus                     584
bps                       584
clean_sheets              584
goals_conceded            584
goals_scored              584
own_goals                 584
penalties_missed          584
penalties_saved           584
red_cards                 584
saves                     584
selected                  584
total_points              584
transfers_balance         584
was_home                  584
yellow_cards              584
opponent_difficulty       584
element                   584
creativity                584
ict_index                 584
influence                 584
threat                    584
value                     584
n6_total_points           584
n6_opponent_difficulty    584
perc_min_played           584
id                        584
element_type              584
dtype: int64

In [29]:
df = df_16_17.append([df_17_18,df_18_19,df_19_20])
#df['element'] = df['element'].apply(str)
df = pd.get_dummies(df,columns=['element_type'])
df_to_export = df
df = df.drop(columns=['name','element','id'], axis=0)
df.dtypes


assists                     int64
bonus                       int64
bps                         int64
clean_sheets                int64
goals_conceded              int64
goals_scored                int64
own_goals                   int64
penalties_missed            int64
penalties_saved             int64
red_cards                   int64
saves                       int64
selected                    int64
total_points                int64
transfers_balance           int64
was_home                    int64
yellow_cards                int64
opponent_difficulty       float64
creativity                float64
ict_index                 float64
influence                 float64
threat                    float64
value                       int64
n6_total_points             int64
n6_opponent_difficulty    float64
perc_min_played           float64
element_type_1              uint8
element_type_2              uint8
element_type_3              uint8
element_type_4              uint8
dtype: object

In [30]:
df.head()

,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,own_goals,penalties_missed,penalties_saved,red_cards,...,influence,threat,value,n6_total_points,n6_opponent_difficulty,perc_min_played,element_type_1,element_type_2,element_type_3,element_type_4
0,0,1,102,2,6,0,0,0,0,0,...,7.2,8.0,53,12,2.500000,1.000000,0,1,0,0
1,1,0,62,2,2,0,0,0,0,0,...,0.0,18.0,55,1,2.166667,0.577778,0,0,1,0
2,0,0,26,0,3,0,0,0,0,0,...,7.2,37.0,76,6,2.400000,0.233333,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,45,0,2.166667,0.000000,0,0,1,0
4,0,0,18,0,6,0,0,0,0,0,...,3.6,18.0,46,7,2.833333,0.333333,0,0,1,0


In [31]:
# Use numpy to convert to arrays

# Labels are the values we want to predict
labels = np.array(df['n6_total_points'])
# Remove the labels from the features
# axis 1 refers to the columns
features = df.drop('n6_total_points', axis = 1)
# Saving feature names for later use
features_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [32]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [33]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (1770, 28)
Training Labels Shape: (1770,)
Testing Features Shape: (591, 28)
Testing Labels Shape: (591,)


In [34]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [35]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'points')

Mean Absolute Error: 5.12 points


In [36]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (sum(errors) / sum(test_labels))
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
##don't use the random forest approach


Accuracy: 39.94 %.


In [37]:
rf.fit(features, labels)

RandomForestRegressor(n_estimators=1000, random_state=42)

In [38]:
predictions = rf.predict(features)

In [39]:
df_to_export['predictions'] = predictions.round()

In [40]:
df_to_export.head()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,own_goals,penalties_missed,penalties_saved,...,value,n6_total_points,n6_opponent_difficulty,perc_min_played,id,element_type_1,element_type_2,element_type_3,element_type_4,predictions
0,Aaron_Cresswell,0,1,102,2,6,0,0,0,0,...,53,12,2.500000,1.000000,454.0,0,1,0,0,13.0
1,Aaron_Lennon,1,0,62,2,2,0,0,0,0,...,55,1,2.166667,0.577778,142.0,0,0,1,0,5.0
2,Aaron_Ramsey,0,0,26,0,3,0,0,0,0,...,76,6,2.400000,0.233333,16.0,0,0,1,0,8.0
3,Aaron_Wan-Bissaka,0,0,0,0,0,0,0,0,0,...,45,0,2.166667,0.000000,612.0,0,0,1,0,0.0
4,Abdoulaye_Doucouré,0,0,18,0,6,0,0,0,0,...,46,7,2.833333,0.333333,482.0,0,0,1,0,7.0


In [41]:
df_to_export.to_csv(r'A:\Documents\20-21 EPL Predictions\16-20 Predictions.csv')

In [44]:
import pulp

# set expected score, prices, positions, clubs, total_budget, sub_factor
# we just need to plug in the new data

##First 4 GW of 2019-20
url = ["https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/gw15.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/gw16.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/gw17.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/gw18.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/gw19.csv",
      "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2020-21/gws/gw20.csv"]

In [46]:
df_n6 = pd.read_excel('A:\Documents\EPL_Fixture_Difficulty.xlsx',sheet_name='Upcoming 6 Games', index_col=0)
df_n6 = pd.DataFrame(df_n6)

df_n6.head(20)

,n6_opponent_difficulty,team_code
team,,
Arsenal,3.333333,1
Aston Villa,3.000000,2
Brighton,3.166667,3
Burnley,3.333333,4
Chelsea,3.333333,5
Crystal Palace,2.500000,6
Everton,3.166667,7
Fulham,3.000000,8
Leicester,3.166667,9


In [47]:
fix_diff = pd.read_excel('A:\Documents\EPL_Fixture_Difficulty.xlsx',sheet_name='20-21', index_col=0)
fix_diff = pd.DataFrame(fix_diff)
fix_diff.head

df_20_21 = pd.DataFrame()
for data in url:
    df = pd.read_csv(data, encoding = "ISO-8859-1")
    df_20_21 = df_20_21.append(df)
    
df_20_21 = df_20_21.merge(fix_diff, left_on='opponent_team', right_on='opponent_team')
df_20_21_agg = df_20_21.groupby(['name'], as_index=False).agg({'assists':'sum',
                                                             'bonus':'sum',
                                                             'bps':'sum',
                                                             'clean_sheets':'sum',                                                             
                                                              'goals_conceded':'sum',
                                                              'goals_scored':'sum',
                                                              'minutes':'sum',
                                                              'own_goals':'sum',
                                                              'penalties_missed':'sum',
                                                              'penalties_saved':'sum',
                                                              'red_cards':'sum',
                                                              'saves':'sum',
                                                              'selected':'sum',
                                                              'total_points':'sum',
                                                              'transfers_balance':'sum',
                                                              'was_home':'sum',
                                                              'yellow_cards':'sum',
                                                              'opponent_difficulty':'mean',
                                                              'round':'count'})

df_20_21_agg['round'] = max(df_20_21_agg['round'])

##Get Last Value for players in GW range
df_20_21_last = df_20_21.groupby('name',as_index=False).last()
df_20_21_last = df_20_21_last[['name','element','position','team','creativity','ict_index','influence','threat','value']]

df_20_21 = df_20_21_agg.merge(df_20_21_last, left_on='name', right_on='name')
df_20_21['perc_min_played'] = df_20_21['minutes']/(df_20_21['round']*90)
df_20_21 = df_20_21.drop(columns=['minutes','round'], axis=0)
df_20_21 = df_20_21.merge(df_n6, left_on='team', right_on='team')
df_20_21.head()
df_20_21 = df_20_21.drop('team', axis=1)

df_20_21['element_type_1'] = np.where(df_20_21['position']== 'GK', 1, 0)
df_20_21['element_type_2'] = np.where(df_20_21['position']== 'DEF', 1, 0)
df_20_21['element_type_3'] = np.where(df_20_21['position']== 'MID', 1, 0)
df_20_21['element_type_4'] = np.where(df_20_21['position']== 'FWD', 1, 0)

df_20_21['element_type'] = 1
df_20_21.loc[df_20_21['position']== 'DEF', 'element_type'] = 2
df_20_21.loc[df_20_21['position']== 'MID', 'element_type'] = 3
df_20_21.loc[df_20_21['position']== 'FWD', 'element_type'] = 4
df_20_21.tail()

,name,assists,bonus,bps,clean_sheets,goals_conceded,goals_scored,own_goals,penalties_missed,penalties_saved,...,threat,value,perc_min_played,n6_opponent_difficulty,team_code,element_type_1,element_type_2,element_type_3,element_type_4,element_type
639,Serge Aurier,0,0,48,0,3,1,0,0,0,...,0.0,52,0.357143,3.166667,17,0,1,0,0,2
640,Sergio ReguilÃ³n,1,3,59,1,2,0,0,0,0,...,25.0,57,0.384127,3.166667,17,0,1,0,0,2
641,Steven Bergwijn,3,0,41,1,5,0,0,0,0,...,0.0,70,0.449206,3.166667,17,0,0,1,0,3
642,Tanguy Ndombele,0,5,104,2,5,2,0,0,0,...,7.0,59,0.644444,3.166667,17,0,0,1,0,3
643,Toby Alderweireld,0,1,31,1,0,1,0,0,0,...,0.0,54,0.142857,3.166667,17,0,1,0,0,2


In [48]:
df_pred = df_20_21.drop(['name','element','position','element_type','team_code'], axis=1)
predictions = rf.predict(df_pred)

In [49]:
df_20_21['predictions'] = predictions.round()
df_20_21.to_csv(r'A:\Documents\20-21 EPL Predictions\20-21 Predictions.csv')
df_20_21.head()

#drop injured players showing up
# df_20_21 = df_20_21[df_20_21['name'] != 'Ãaglar SÃ¶yÃ¼ncÃ¼'] 
# df_20_21 = df_20_21[df_20_21['name'] != 'Kevin De Bruyne'] 


expected_scores = df_20_21['predictions']
prices = df_20_21['value']/10
positions = df_20_21['element_type']
clubs = df_20_21['team_code']

names = df_20_21['name']

In [52]:

def select_team(expected_scores, prices, positions, clubs, total_budget=99.8, sub_factor=0.2):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 1) == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 1) == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 2) <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 2) == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 3) <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 3) == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 4) <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 4) == 3

    # club constraint
    for club_id in np.unique(clubs):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if clubs[i] == club_id) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

In [53]:
decisions, captain_decisions, sub_decisions = select_team(expected_scores.values, prices.values, positions.values, clubs.values)
# print results
for i in range(df.shape[0]):
    if decisions[i].value() != 0:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))
print()
print("Subs:")
# print results
for i in range(df.shape[0]):
    if sub_decisions[i].value() == 1:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))

print()
print("Captain:")
# print results
for i in range(df.shape[0]):
    if captain_decisions[i].value() == 1:
        display(Markdown("**{}** Points = {}, Price = {}".format(names[i], expected_scores[i], prices[i])))

Total expected score = 362.99999999999994


**Michail Antonio** Points = 23.0, Price = 6.3

**Bruno Miguel Borges Fernandes** Points = 29.0, Price = 11.1

**Pedro Lomba Neto** Points = 25.0, Price = 6.0

**Willy Boly** Points = 22.0, Price = 5.4

...

**Bernd Leno** Points = 22.0, Price = 4.9

**Kieran Tierney** Points = 22.0, Price = 5.3

**Kyle Walker-Peters** Points = 20.0, Price = 4.8


Captain:


**Ederson Santana de Moraes** Points = 35.0, Price = 6.0